In [2]:
import os
from tqdm import tqdm
import pandas as pd
import txt_to_prob as tp

# 각 class 별, subject ID read
print("Load subject's ID")
# print('-- Read PSG_list2.xlsx')
df_subject_ID = pd.read_csv('D:\\USC\\code_mine\\txt_to_prob\\subject_info\\PSG_list_by_CYS.csv')

pd_healthy = df_subject_ID.iloc[:,0].dropna()
pd_OSA = df_subject_ID.iloc[:,1].dropna()
pd_INS = df_subject_ID.iloc[:,2].dropna()
pd_COMISA = df_subject_ID.iloc[:,3].dropna()

pd_total = pd.concat([pd_healthy, pd_OSA, pd_INS, pd_COMISA])
print("-- Number of subjects: Healthy({}), OSA({}), Insomnia({}), COMISA({})"
      .format(len(pd_healthy), len(pd_OSA), len(pd_INS), len(pd_COMISA)))
print("-- Total: {}".format(len(pd_total)))

# path_txt = 'E:\\samsung_original\\ensamble_test' # txt 파일이 저장되어 있는 경로/
path_txt = 'E:\\samsung_original\\whole_data_txt\\data3' # txt 파일이 저장되어 있는 경로
txt_subjects = pd.Series(os.listdir(path_txt)) # 모든 txt 파일의 filename이 담겨 있는 pd.Series

df_SOL = pd.read_csv('subject_info\\all_class.csv', index_col=0)

print('Number of txt files: {}'.format(txt_subjects.size))

# 이미 probabilistic hypnogram이 생성된 subject list
# path_prob_hyp = 'E:\\probabilistic_hypnogram'
path_prob_hyp = 'E:\\samsung_original\\whole_data_txt\\data3_prob'
list_hyp_exist = os.listdir(path_prob_hyp)

path_save = 'E:\\samsung_original\\whole_data_txt\\data3_prob'

Load subject's ID
-- Number of subjects: Healthy(1283), OSA(1933), Insomnia(727), COMISA(785)
-- Total: 4728
Number of txt files: 690


In [3]:
QNAN = []
for sub_ID in tqdm(pd_total,desc='Processing data1 (n=1499) ... '):
      sub_ID = sub_ID.upper()
      if sub_ID in list_hyp_exist:
            # 이미 probabilistic hypnogram이 있는 경우, pass !
      #     print('Already processed subject ... ')
            continue 
      
      # Read txt files and extract eeg data for all channels
      txt2np = tp.txt2np(path_txt, txt_subjects, sub_ID)
      txt2np.txt_filenames()

      if len(txt2np.temp_txt_subjects) < 2:
            continue
      np_all_eeg = txt2np.read_txt()

      if np_all_eeg.shape[0] == 0:
            # txt 파일이 비어 있는 경우 pass
            continue

      QNAN.append(txt2np.QNAN_dic)

      # Create mne.raw instance 
      # - data: np_all_eeg
      # - raw.info['subject_info']['his_id']: subject ID
      # - preprocessing: re_referencing + cropping
      # - raw.ch_names: ['F3-A2', 'F4-A1', 'C3-A2', 'C4-A1', 'O1-A2', 'O2-A1']

      SOL = df_SOL.loc[sub_ID, 'Sleep latency (min)']
      np2raw = tp.np2raw(sub_ID, SOL)
      np2raw.np2raw(np_all_eeg)
      # len_np - SOL < 330 인 경우, pass
      if (np_all_eeg.shape[1]/np2raw.temp_sfreq/60 - SOL) < 330:
            continue
      np2raw.re_ref()
      raw_re_ref_cropped = np2raw.raw_cropping()

      # Automatic Sleep Staging
      # - apply ensamble using all six eeg channels
      # - automatiaclly saved
      #   current directory
      #   -- hypnograms
      #   -- -- subject_1
      #   -- -- subject_2
      #   -- -- ...
      #   -- -- subject_n
      #   -- -- -- predicted_hypnogram.csv
      #   -- -- -- probabilistic_hypnogram.csv
      automatic_staging = tp.automatic_staging()
      automatic_staging.get_hypnos_and_probs(raw_re_ref_cropped.copy())
      automatic_staging.ensamble_stagig(path_save)

Processing data1 (n=1499) ... : 100%|██████████| 4728/4728 [1:17:25<00:00,  1.02it/s]  


In [4]:
df_final_dic = pd.DataFrame(columns = ['A1', 'C4'])
for temp_dic in QNAN:
    if len(temp_dic.keys()) != 0:
        temp_sub_ID = list(temp_dic.keys())[0]
        df_final_dic.loc[temp_sub_ID] = [0, 0]
        if list(temp_dic.values())[0] == 'A1':
            df_final_dic.loc[temp_sub_ID, 'A1'] = 1
        elif list(temp_dic.values())[0] == 'C4':
            df_final_dic.loc[temp_sub_ID, 'C4'] = 1


In [5]:
print("Number of QNAN in A1 \n{}".format(df_final_dic['A1'].value_counts()))
print("\nNumber of QNAN in C4 \n{}".format(df_final_dic['C4'].value_counts()))
df_final_dic

Number of QNAN in A1 
1    138
Name: A1, dtype: int64

Number of QNAN in C4 
0    138
Name: C4, dtype: int64


,A1,C4
PE150319,1,0
PE150332,1,0
PE150352,1,0
PE150400,1,0
PE150416,1,0
...,...,...
PE150682,1,0
PE150687,1,0
PE150692,1,0
PE150695,1,0


In [6]:

df_final_dic.to_csv('D:\\USC\\code_mine\\txt_to_prob\\QNaN_inspection\\data3_QNaN.csv')